In [1]:
import sys
sys.path.append('..')

import constants.file_handler_constants as fh
from constants.user_constants import *
from constants.attraction_constants import *
from packages.user_preference_prompt_by_persona.gernerate_preference import *

import os
import glob
import time
import pandas as pd
import numpy as np
import json
import requests
import ast
import random as rd


In [2]:
# keep save id by following result to constant (manually)

user_id_love_nature = []
user_id_love_art = []

try:
    API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/user/getAllUser"
    headers = {
        "Authorization": f"{os.environ['AUTH_BRANCH_API_KEY']}",
        "Content-Type": "application/json"
    }
    res_all_user = requests.get(url=API_ENDPOINT, headers=headers).json()
    user_id_love_nature = [cur_user['_id'] for cur_user in res_all_user if ("love_nature_" in cur_user['username'])] 
    user_id_love_art = [cur_user['_id'] for cur_user in res_all_user if ("love_art_" in cur_user['username'])] 
   
except Exception as e:
    print("get user failed !")
    print(e)

print("user_id_love_nature :\n", user_id_love_nature)
print("user_id_art_nature :\n", user_id_love_art)


user_id_love_nature :
 ['677e41b2519209b0aaff4d76', '677e41b6519209b0aaff4d7b', '677e41b9519209b0aaff4d80', '677e41bc519209b0aaff4d85', '677e41bf519209b0aaff4d8a', '677e41c2519209b0aaff4d8f', '677e41c5519209b0aaff4d94', '677e41c8519209b0aaff4d99', '677e41cb519209b0aaff4d9e', '677e41ce519209b0aaff4da3', '677e41d1519209b0aaff4da8', '677e41d4519209b0aaff4dad', '677e41d7519209b0aaff4db2', '677e41da519209b0aaff4db7', '677e41de519209b0aaff4dbc', '677e41df519209b0aaff4dc1', '677e41e1519209b0aaff4dc6', '677e41e2519209b0aaff4dcb', '677e41e3519209b0aaff4dd0', '677e41e5519209b0aaff4dd5']
user_id_art_nature :
 ['677e4cd4519209b0aaff4f8c', '677e544b519209b0aaff4fd6', '677e544e519209b0aaff4fdb', '677e5451519209b0aaff4fe0', '677e5454519209b0aaff4fe5', '677e5457519209b0aaff4fea', '677e545a519209b0aaff4fef', '677e545d519209b0aaff4ff4', '677e5460519209b0aaff4ff9', '677e5464519209b0aaff4ffe', '677e5467519209b0aaff5003', '677e5469519209b0aaff5008', '677e546d519209b0aaff500d', '677e5470519209b0aaff5012', '

In [4]:
''' 
20 rating of each user in  following persona have been created:
- user_persona_nature --> love_art_{nth} -------> not yet
- user_persona_temple_historical --> love_temple_historical_{nth} -------> not yet
'''

# select both the corresponding group of user_id and attraction_id in 'user_constants'
selected_persona_users_id = USER_ID_LOVE_ART
selected_persona_attractions_id = ATTRACTION_ID_LOVE_ART

for cur_user_id in selected_persona_users_id:
    for cur_attraction_id in selected_persona_attractions_id:
        try:
            API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/userrating/patch/{cur_user_id}"
            cur_payload = {
                "attractionId": cur_attraction_id, 
                "rating_score": rd.randint(4, 5)
            }
            res_user_rating = requests.patch(url=API_ENDPOINT, json=cur_payload).json()
            
        except Exception as e:
            print("get user_rating failed !")
            print(e)
